In [88]:
# Point 5: Deeper analysis of the data

import pandas as pd
import numpy as np

top_2_array = set()
top_5_array = set()
top_10_array = set()

bottom_2_array = set()
bottom_5_array = set()
bottom_10_array = set()

data = pd.read_csv('data/x_train_gr_smpl.csv')
for i in range(10):
    data = data.reindex(np.arange(data.shape[0]))
    labels = pd.read_csv(f'data/y_train_smpl_{i}.csv')
    data['label'] = labels
    data = data.sample(frac=1)
    
    corr_label = data.drop("label", axis=1).apply(lambda x: x.corr(data.label))
    corr_label = [(index, abs(corr_val), i) for index, corr_val in enumerate(corr_label)]
    corr_label = sorted(corr_label, key=lambda tup: tup[1], reverse=True)  # Order by correlation value
            
    for i, tup in enumerate(corr_label[:10]):
        if i < 2:
            top_2_array.add(tup[0])
        if i < 5:
            top_5_array.add(tup[0])
        if i < 10:
            top_10_array.add(tup[0]) #, tup[2]))
            
    for i, tup in enumerate(corr_label[-10:]):
        if i < 2:
            bottom_2_array.add(tup[0])
        if i < 5:
            bottom_5_array.add(tup[0])
        if i < 10:
            bottom_10_array.add(tup[0])

In [89]:
# a = sorted([(int(round(x/48)), x % 48) for x in bottom_2_array])
# a = a.filter(lambda (x, y): x >10 or x < 37)
# print(a)
# b = sorted([x % 48 for x in bottom_5_array])
# print(b)
# c = sorted([(int(round(x[0]/48)), x[0] % 48, x[1]) for x in top_10_array])
# print(c)

In [90]:
# Point 6: Try to improve the classification

data = pd.read_csv('data/x_train_gr_smpl.csv')
labels = pd.read_csv('data/y_train_smpl.csv')

data_top_2 = data[data.columns[list(top_2_array)]].copy(deep=True)
data_top_2['label'] = labels
# data_top_2 = data_top_2.sample(frac=1)

data_top_5 = data[data.columns[list(top_5_array)]].copy(deep=True)
data_top_5['label'] = labels
# data_top_5 = data_top_5.sample(frac=1)

data_top_10 = data[data.columns[list(top_10_array)]].copy(deep=True)
data_top_10['label'] = labels
# data_top_10 = data_top_10.sample(frac=1)

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

X = data_top_2.iloc[:, :-1]  # All columns but the label
y = data_top_2['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.21      0.37      0.27       462
           1       0.25      0.18      0.21       614
           2       0.12      0.90      0.21       137
           3       0.92      0.54      0.68       427
           4       0.58      0.41      0.48       723
           5       0.85      0.47      0.61       665
           6       0.50      0.62      0.55       271
           7       0.22      0.33      0.27        89
           8       0.34      0.06      0.10       681
           9       0.22      0.44      0.29       109

    accuracy                           0.37      4178
   macro avg       0.42      0.43      0.37      4178
weighted avg       0.49      0.37      0.38      4178



In [92]:
X = data_top_5.iloc[:, :-1]
y = data_top_5['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.39      0.30       475
           1       0.28      0.20      0.24       601
           2       0.12      0.86      0.20       150
           3       0.93      0.59      0.72       408
           4       0.55      0.33      0.41       741
           5       0.92      0.46      0.61       742
           6       0.49      0.65      0.56       257
           7       0.14      0.38      0.21        78
           8       0.62      0.09      0.16       627
           9       0.24      0.47      0.32        99

    accuracy                           0.37      4178
   macro avg       0.46      0.44      0.37      4178
weighted avg       0.56      0.37      0.40      4178



In [93]:
X = data_top_10.iloc[:, :-1]
y = data_top_10['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.38      0.30       478
           1       0.30      0.19      0.23       635
           2       0.12      0.92      0.22       149
           3       0.94      0.61      0.74       445
           4       0.65      0.37      0.47       678
           5       0.81      0.57      0.67       703
           6       0.53      0.68      0.59       246
           7       0.16      0.49      0.24        72
           8       0.67      0.13      0.21       667
           9       0.55      0.55      0.55       105

    accuracy                           0.41      4178
   macro avg       0.50      0.49      0.42      4178
weighted avg       0.58      0.41      0.43      4178



# 7. Conclusions (Draft)

### Which streets signs are harder to recognise?
Based on the recall measure labels 1 and 8 are the harsdest to recognize.
### Which street signs are most easily confused?
TODO: Print confusion matrix
### Which attributes (fields) are more reliable and which are less reliable in classification of street signs?
By looking at top_2_array we can identify the most reliable features (pixels). By doing some operations like the module we can see those pixels are closer to the centre of the image (or at least don't belong to the borders).
Regarding the less reliable pixels we consider the ones with the lowest correlation value with respect to the label.
### What was the purpose of Tasks 5 and 6?
Prepare the data in a way that we keep the most relevant attributes.
### What would happen if the data sets you used in Tasks 4, 5 and 6 were not randomised?
TODO: Rethink. Guess: NO VISIBLE CHANGES
### What would happen if there is cross-correlation between the non-class attributes?
There is actually cross-correlation between pixels that are close to each other. As a consecuence of that, if two pixels are correlated (adjacent) to each other, and one of them is included in the top 10 of the most correlated pixels with the label, the other is likely to be in the top 10 as well.